In [1]:
import os, random, sys, argparse
# sys.path.append('/DATA/xaiva/model/line3/tensorrt_workspace/yolox/yolox')
import torch, cv2, glob, numpy as np
import torch.backends.cudnn as cudnn

from yolox.exp import get_exp
from yolox.data import ValTransform
from yolox.utils import (
    postprocess
)
from torch2trt import torch2trt


In [2]:
exp_name = 'yolox-m'
exp = get_exp(None, exp_name)
model = exp.get_model().eval().cuda()
weights = 'yolox_m.pth'

In [3]:
ckpt = torch.load(weights, map_location='cuda')
model.load_state_dict(ckpt['model'])
model.to('cuda').eval()
print ('load model..')

load model..


In [4]:
x = torch.ones((1, 3, 640, 640)).to(torch.device('cuda'))
model.head.decode_in_inference = False

In [5]:
print ('convert trt..')
model_trt = torch2trt(
        model,
        [x],
        fp16_mode=True,
        max_batch_size=24,
    )

convert trt..
max_batch_size 24
self.engine <tensorrt.tensorrt.ICudaEngine object at 0x7f5f54dc9970>


TypeError: a bytes-like object is required, not 'tensorrt.tensorrt.ICudaEngine'

In [ ]:
print ('save trt..')
output_path = 'yolox_m_ij.trt'
torch.save(model_trt.state_dict(), output_path)